In [1]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 975.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.2 MB

In [2]:
import gradio as gr
import os
import pandas as pd
from transformers import pipeline
import torch
import scipy
import numpy as np

Mounted at /content/drive


In [3]:
path = 'Datasets/Melody Script'
melody_path = 'Datasets/Melodies'

In [4]:
files = os.listdir(path)
files = [ f for f in files if f.endswith('.pkl')]
films = [f.split('.pkl')[0] for f in files]

In [5]:
films

['Avatar',
 'Avengers,-The-(2012)',
 'Pulp-Fiction',
 'Star-Wars-The-Phantom-Menace',
 'Star-Wars-The-Force-Awakens',
 'Star-Wars-Attack-of-the-Clones',
 'Star-Wars-A-New-Hope',
 'Shrek-the-Third',
 'Star-Wars-Return-of-the-Jedi',
 'Star-Wars-The-Empire-Strikes-Back',
 'V-for-Vendetta',
 'Toy-Story',
 'Titanic']

# Caricamento del modello MusicGen

In [6]:
synthesiser = pipeline("text-to-audio", "facebook/musicgen-large", torch_dtype=torch.float16, device = 0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/98.1k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.73G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

# Definizione funzioni Callback Gradio

In [7]:
def erase_tag( string ):
  if '<eos>' in string:
    return string.split('<eos>')[0]
  else:
    return string

In [8]:
def update_scenes(film):
    name_file = film+".pkl"
    global df
    df = pd.read_pickle(path+"/"+name_file)
    scenes = list(df['Scene_Names'].unique())
    #print(scenes)
    return gr.Dropdown(choices=scenes, value=None, interactive=True)

In [9]:
def update_description(scene):
    if scene is None:
        return "Title", "Emotion", "Music Genre", ""
    scene = df[df['Scene_Names'] == scene]

    genre = scene['genre'].iloc[0]
    title = scene['title'].iloc[0]
    emotion = scene['emotion'].iloc[0]
    scene_description = scene['Contents'].iloc[0]

    genre = erase_tag(genre)
    title = erase_tag(title)
    emotion = erase_tag(emotion)

    return "Title: " + title, "Emotions: " + emotion,"Music Genre: " + genre, scene_description

In [10]:
def generate_audio(film, scene):
    scene = df[df['Scene_Names'] == scene]

    genre = scene['genre'].iloc[0]
    title = scene['title'].iloc[0]
    emotion = scene['emotion'].iloc[0]

    genre = erase_tag(genre)
    title = erase_tag(title)
    emotion = erase_tag(emotion)

    prompt = "%s %s"%(genre,emotion)

    name_file = str(scene.index[0]) + "__" + film + '.wav'
    save_path = os.path.join(melody_path,name_file)

    music = synthesiser(prompt, forward_params={"do_sample": True, 'max_new_tokens' : 1500})
    scipy.io.wavfile.write(save_path, rate=music["sampling_rate"], data=music["audio"])

    return gr.Audio(save_path)

# Definizione dell'interfaccia Gradio

In [11]:
scenes = []
df = pd.DataFrame()

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            film_dropdown = gr.Dropdown(label="Seleziona un Film", choices=films, interactive=True)
            scene_dropdown = gr.Dropdown(label="Seleziona una Scena", choices=[], interactive=False)
            generate_button = gr.Button("Genera Audio")
        with gr.Column():
            audio_output = gr.Audio(label="Melodia Generata")
            titolo = gr.Markdown("Title")
            emotion = gr.Markdown("Emotion")
            genre = gr.Markdown("Genre")

    with gr.Row():
        scene_description = gr.Label("", label="Scene Description")


    # Gestione degli eventi
    film_dropdown.change(fn=update_scenes, inputs=film_dropdown, outputs=scene_dropdown)
    scene_dropdown.change(fn=update_description, inputs=scene_dropdown, outputs=[titolo,emotion,genre, scene_description])
    generate_button.click(fn=generate_audio, inputs=[film_dropdown, scene_dropdown], outputs=audio_output)

In [12]:
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8b9ac50a2e9e28cc6c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
